# Tune Hyperparameters

There are many machine learning algorithms that require *hyperparameters* (parameter values that influence training, but can't be determined from the training data itself). For example, when training a logistic regression model, you can use a *regularization rate* hyperparameter to counteract bias in the model; or when training a convolutional neural network, you can use hyperparameters like *learning rate* and *batch size* to control how weights are adjusted and how many data items are processed in a mini-batch respectively. The choice of hyperparameter values can significantly affect the performance of a trained model, or the time taken to train it; and often you need to try multiple combinations to find the optimal solution.

In this case, you'll train a classification model with two hyperparameters, but the principles apply to any kind of model you can train with Azure Machine Learning.

## Install the Azure Machine Learning SDK

The Azure Machine Learning SDK is updated frequently. Run the following cell to upgrade to the latest release, along with the additional package to support notebook widgets.

In [1]:
!pip install --upgrade azureml-sdk azureml-widgets

## Connect to your workspace

With the latest version of the SDK installed, now you're ready to connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.57.0 to work with coursera


## Prepare data

In this lab, you'll use a dataset containing details of diabetes patients. Run the cell below to create this dataset (if it already exists, the existing version will be used)

In [3]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


## Prepare a training script

Now let's create a folder for the training script you'll use to train the model.

In [4]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

Folder ready.


Now create the Python script to train the model. In this example, you'll use a *Gradient Boosting* algorithm to train a classification model. The script must include:

- An argument for each hyperparameter you want to optimize (in this case, the learning rate and number of estimators for the Gradient Boosting algorithm)
- Code to log the performance metric you want to optimize for (in this case, you'll log both AUC and accuracy, so you can choose to optimize the model for either of these)

In [5]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Input dataset
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# Hyperparameters
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# Add arguments to args collection
args = parser.parse_args()

# Log Hyperparameter values
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a Gradient Boosting classification model with the specified hyperparameters
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes_training-hyperdrive/diabetes_training.py


## Create compute

Hyperparameter tuning involves running multiple training iterations with different hyperparameter values and comparing the performance metrics of the resulting models. To do this efficiently, we'll take advantage of on-demand cloud compute and create a cluster - this will allow multiple training iterations to be run concurrently.

Use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist).

> **Important**: Change *your-compute-cluster* to the name of your compute cluster in the code below before running it! Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


## Run a hyperparameter tuning experiment

Azure Machine Learning includes a hyperparameter tuning capability through *hyperdrive* experiments. These experiments launch multiple child runs, each with a different hyperparameter combination. The run producing the best model (as determined by the logged target performance metric for which you want to optimize) can be identified, and its trained model selected for registration and deployment.

> **Mote**: In this example, we aren't specifying an early stopping policy. Such a policy is only relevant if the training script performs multiple training iterations, logging the primary metric for each iteration. This approach is typically employed when training deep neural network models over multiple *epochs*.

In [7]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed (we need scikit-learn, Azure ML defaults, and Azure ML dataprep)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=sklearn_env,
                                compute_target = training_cluster)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# Run the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

2024-10-13 17:34:17.832275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 17:34:18.558627: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 17:34:18.782671: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 17:34:20.458085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-13 17:34:24.668457: W tensorflow/comp

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0',
 'target': 'your-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-10-13T17:33:58.753765Z',
 'endTimeUtc': '2024-10-13T17:49:05.257466Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e262907d-08d7-4d46-b683-13bef22ead86',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'space_size': '6',
  'best_child_run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_3',
  'score': '0.9885804604667666',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlCl

Let's go through this piece of code step-by-step - 

### Import Required Libraries

```python
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails
```

Here, you're importing several key libraries from the Azure Machine Learning SDK. These libraries will help us set up and run an experiment for **hyperparameter tuning**.

- **Experiment**: This helps manage and track your machine learning experiments. An experiment is a container for all the runs (training jobs) you do with different settings.
- **ScriptRunConfig**: This is used to configure and run your Python training script in Azure, specifying the environment, script to run, and where to run it.
- **Environment**: This allows you to define the Python environment (dependencies and packages) your training script needs.
- **GridParameterSampling**: This helps us define a grid search (a type of hyperparameter search) for trying different combinations of hyperparameter values.
- **HyperDriveConfig**: This is the configuration for setting up a hyperparameter tuning experiment (in Azure ML, it's called **HyperDrive**).
- **PrimaryMetricGoal**: This defines the goal for optimizing your model (for example, maximizing or minimizing a specific metric like accuracy or AUC).
- **choice**: This is a helper function to specify the range of values to try for a hyperparameter.
- **RunDetails**: A widget to visually track the status of your experiment within the notebook.

### Define Hyperparameter Sampling

```python
params = GridParameterSampling({
    '--learning_rate': choice(0.01, 0.1, 1.0),
    '--n_estimators': choice(10, 100)
})
```

**What are hyperparameters?**

In machine learning, hyperparameters are settings that you define before training a model, like:
- **Learning rate**: Controls how much the model adjusts weights with each step during training.
- **Number of estimators**: Specifies the number of trees in an ensemble model (like Gradient Boosting).

Here, you are defining a **grid search** for hyperparameters. In this case:
- You're telling Azure ML to try different values of `learning_rate`: 0.01, 0.1, and 1.0.
- You're also specifying `n_estimators` to take values of 10 or 100.

The system will run multiple training jobs, each using a different combination of these hyperparameters to find the best-performing model.

### Configure HyperDrive

```python
hyperdrive = HyperDriveConfig(run_config=script_config,
                              hyperparameter_sampling=params,
                              primary_metric_name='AUC',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=6,
                              max_concurrent_runs=2)
```

This block sets up the **HyperDrive experiment**. Let's break it down:

- **run_config**: This refers to the `ScriptRunConfig` object (not shown in this snippet). It contains details about the Python script that trains the model, the compute environment, and other configurations needed to run the training.
- **hyperparameter_sampling=params**: Here, we pass the hyperparameters we defined above (`learning_rate` and `n_estimators`). This tells HyperDrive to try the different combinations of these hyperparameters.
- **primary_metric_name='AUC'**: When training the model, we calculate different metrics (e.g., accuracy, precision, etc.). Here, we're using **AUC** (Area Under the Curve) as the main metric to evaluate model performance.
- **primary_metric_goal=PrimaryMetricGoal.MAXIMIZE**: We are telling the system that we want to maximize the AUC score (the higher, the better).
- **max_total_runs=6**: We limit the total number of hyperparameter combinations to try. With 3 values for `learning_rate` and 2 for `n_estimators`, we could try up to 6 combinations.
- **max_concurrent_runs=2**: This means the system will run 2 training jobs in parallel to save time.

### Run the Experiment

```python
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)
```

- **Experiment**: You create an experiment object and give it a name (`'mslearn-diabetes-hyperdrive'`). The **workspace** (`ws`) is where your experiment is stored and tracked within Azure.
- **experiment.submit(config=hyperdrive)**: This submits the HyperDrive experiment for execution. Azure ML will now start training the model with different hyperparameter combinations, as defined in `hyperdrive`.

### Track the Experiment

```python
RunDetails(run).show()
run.wait_for_completion()
```

- **RunDetails(run).show()**: This displays a widget in the notebook that shows the status of the runs. You'll see updates as each combination of hyperparameters is tried, and you can track the metrics for each run.
- **run.wait_for_completion()**: This tells the notebook to wait until all runs have finished before proceeding. Azure ML will try all 6 combinations (or fewer, depending on the configuration) and report the results.

### Summary

This code sets up and runs an experiment to automatically search for the best hyperparameter values for training a model. Here's how the whole process works:

1. **Hyperparameters**: You're trying different values of `learning_rate` and `n_estimators` to see which combination produces the best model.
2. **Experiment**: Azure ML runs the training job multiple times, each with different hyperparameter settings.
3. **Results**: As the experiment runs, you'll see the results, and Azure will show you which combination of hyperparameters gives the best performance (according to the AUC metric).

This makes it easier to optimize models without manually running each possible combination yourself.

You can view the experiment run status in the widget above. You can also view the main Hyperdrive experiment run and its child runs in [Azure Machine Learning studio](https://ml.azure.com).

> **Note**: If a message indicating that a non-numeric can't be visualized is displayed, you can ignore it.

## Determine the best performing run

When all of the runs have finished, you can find the best one based on the performance metric you specified (in this case, the one with the best AUC).

In [8]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_3', 'hyperparameters': '{"--learning_rate": 0.1, "--n_estimators": 100}', 'best_primary_metric': 0.9885804604667666, 'status': 'Completed'}
{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_5', 'hyperparameters': '{"--learning_rate": 1.0, "--n_estimators": 100}', 'best_primary_metric': 0.9857545250800522, 'status': 'Completed'}
{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_4', 'hyperparameters': '{"--learning_rate": 1.0, "--n_estimators": 10}', 'best_primary_metric': 0.982908128731084, 'status': 'Completed'}
{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_1', 'hyperparameters': '{"--learning_rate": 0.01, "--n_estimators": 100}', 'best_primary_metric': 0.9559393638830617, 'status': 'Completed'}
{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8f58540ec0_2', 'hyperparameters': '{"--learning_rate": 0.1, "--n_estimators": 10}', 'best_primary_metric': 0.9516323866285732, 'status': 'Completed'}
{'run_id': 'HD_f2a483ea-56e8-4b79-9e29-4a8

Now that you've found the best run, you can register the model it trained.

In [9]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 7
	 Training context : Hyperdrive
	 AUC : 0.9885804604667666
	 Accuracy : 0.9457777777777778


diabetes_model version: 6
	 Training context : Inline Training
	 AUC : 0.8746430170655255
	 Accuracy : 0.889


diabetes_model version: 5
	 Training context : Inline Training
	 AUC : 0.8806060002985222
	 Accuracy : 0.894


diabetes_model version: 4
	 Training context : Inline Training
	 AUC : 0.8788322802129458
	 Accuracy : 0.891


diabetes_model version: 3
	 Training context : Pipeline
	 AUC : 0.8867338641533674
	 Accuracy : 0.9008888888888889


diabetes_model version: 2
	 Training context : Pipeline
	 AUC : 0.8850872543910331
	 Accuracy : 0.9


diabetes_model version: 1
	 Training context : Compute cluster
	 AUC : 0.88375696004516
	 Accuracy : 0.8986666666666666


amlstudio-predict-penguin-clus version: 1
	 CreatedByAMLStudio : true


amlstudio-predict-diabetes version: 1
	 CreatedByAMLStudio : true


amlstudio-predict-auto-price-1 version: 1
	 CreatedByAMLStudio : tr

> **More Information**: For more information about Hyperdrive, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters).